## Quickstart

*Quibbler* allows creating highly interactive data analysis applications, using completely standard *Python*, *NumPy* and *Matplotlib* programming syntax. There is therefore very little to learn to get started.

This page starts with a "minimal-demo" to quickly get you up and running. 

For a more data-oriented quick demo, see [[Simple demo]], and consider also the [[Examples]]. 

For a more methodological tour, see [[Introduction]] and the [[User Guide]]. 

Before starting, please first install `pyquibbler` (see [[Installation]]).

### Import

`pyquibbler` is conventionally imported as `qb`. In addition, it is convenient to specifically import some often-used functions such as `iquib` (which will be explained below). Following import, we need to execute `qb.override_all()` which configures *NumPy* and *Matplotlib* functions to work with *Quibbler*. A typical import therefore looks as follows:

In [1]:
# Quibbler import:
import pyquibbler as qb
from pyquibbler import iquib
qb.override_all()

# Other imports:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

### Minimal demo

Here is a very simple example of a *Quibbler* app, which creates a figure with a draggable marker and matching text:

In [2]:
xy = iquib(np.array([1., 1.]))
x = xy[0]
y = xy[1]
plt.axis([0, 2, 0, 2])
plt.plot(x, y, marker='o', picker=True)
plt.text(x, y + 0.05, np.array2string(xy, precision=2));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
plt.axis('square')
plt.axis([0, 2, 0, 2])

(0.0, 2.0, 0.0, 2.0)

As you can see, except for the use of the function `iquib` (which we will explain below), the rest is a completely standard *Python* code for plotting a marker at position x=1, y=1 and adding next to it a text label with the string of these coordinates. Indeed, running this code plots the marker and the text as classically expected. Yet, unlike this standard behavior, here the data items and graphics are all bi-directionally linked. First, assigning new values to the upstream variable `xy` immediately moves the marker and refreshes the text to reflect the change. Second, the plotted marker can be dragged and as we drag it, the position and label of the text automatically change.

[[/images/Quickstart_xy_1_1_drag.gif]]

In *Quibbler*, thereby, we easily get interactive functionality using completely standard programming syntax and without worrying at all about the tedious programming of event-specific callback functions otherwise needed for creating interactive behaviors.

### How does it work?

Below, we briefly explain the above example, while providing a more general view of *Quibbler* functionality.

#### The quib object
*Quibbler* functionality is based on the *quib* object. The quib is an object that represents an output *value* as well as the *function* and *arguments* used to calculate this value. There are two major types of quibs: input-quibs (i-quibs) which take a regular *Python* object as their argument and present it as their value, and function-quibs (f-quibs) that calculate their output value by applying a given function to a given list of arguments, which could include other quibs and any other *Python* objects. 

#### Input-quibs
Input-quibs are created using the function `iquib`, which transforms any regular *Python* object into an i-quib. In our case `xy = iquib(np.array([1., 1.]))` creates an i-quib `xy` whose value is *NumPy* array `[1., 1.]`. 

#### Function-quibs
Function-quibs are then created naturally whenever we use quibs as part of standard expressions or functions. Indeed, *Quibbler* modifies standard functions and operators such that they can work directly with quibs. Such Quibbler-supported functions, also called _quiby functions_, include not only many standard *Python*, *NumPy* and *Matplotlib* functions (see [full list[Quiby functions]]), but also operators (such as `+`, `-`, `<`, `>`, `**`, `@`), and array indexing. We can therefore easily define a chained network of function quibs using standard *Python* syntax. 

In our case, when our code executes `x = xy[0]` and `y = xy[1]` it creates the f-quibs `x` and `y` whose function is to reference the quib `xy` at positions 0 and 1, respectively. Next, the command `plt.plot(x, y, ...)` defines an f-quib whose function is to perform `plt.plot` on the values of `x` and `y`. Similarly, `y + 0.05` is a function quib that adds 0.05 to the value of `y`, `np.array2string(xy, ...)` is a function-quib that performs the `array2string` on the value of `xy` and, finally, `plt.text(...)` is a function quib that calls `plt.text` with the values of its quib arguments. 

#### Upstream changes automatically propagate to affect downstream results
All of these quibs are created *declaratively*: they are functional objects whose value changes upon upstream changes. Thereby, when we assign `xy[1] = 0.5`, this changes `y`, which in turn changes the plot and the text. 

#### Interaction with the graphics is inverse-propagated to a change upstream quib values
The relation above can also go backwards. Changing the graphics by dragging the marker is translated into an assignment to the corresponding arguments of the plt.plot function, the `x` and `y` quibs. Since `x` and `y` are functional quibs, the assignment is further inverse-propagated upstream to the i-quib `xy` where it is actualized. The change in `xy` then percolates downstream to affect the text position and text label.

### A data-oriented example

The above principles can be used to readily create fun and interactive analysis of data, where our result depend on several paramters that we need to choose. 

Consider making a polynomial fit to "predict" the stock values. We will need to specify a window of data where we fit the model, and the polynomial degree. 

In *Quibbler*, by simply plotting the value of these choices we immediately make the analysis interactive. 

Furthermore, we can combine quibs with Matplotlib.widgets to readily make changes interactive.

Consider the following code: